As the eigenvector-field is characterized by inherent orientational discontinuities, we need to account for them at each integration step. We do so by introducing an appropriate rescaling that alters the tensorline ODE in a fashion so that its numerical solution yields a globally smooth set of tensorlines. This type of rescaling was first proposed by [1].

The original ODE for a tensorline $ \mathbf{x}(s) $ associated to the eigenvector field $ \mathbf{\xi}_i $ is given by:

\begin{equation}
\dfrac{d}{ds}\mathbf{x}(s) = \xi_{i}(s)
\label{eq: ODE_1}
\tag{1}
\end{equation}


[2] proposed rewriting the ODE \ref{eq: ODE_1} as:

\begin{equation}
\dfrac{d}{ds}\mathbf{x}(s) = \mathrm{sign}(\langle \xi_i(s), \xi_i(s-\Delta s)\rangle) \alpha(\mathbf{x}(s)) \xi_i(s),
\label{eq: ODE_2}
\tag{2}
\end{equation} where $ \alpha(\mathbf{x}(s)) = \dfrac{(\lambda_2(\mathbf{x}(s))-\lambda_1(\mathbf{x}(s))^2}{(\lambda_2(\mathbf{x}(s))+\lambda_1(\mathbf{x}(s))^2}$. Tensorline singularities are thus rescaled to fixed points of eq. \ref{eq: ODE_2} as $ \lambda_2(\mathbf{x}(s)) = \lambda_1(\mathbf{x}(s)) $ holds at such points.
For incompressible velocity fields it holds $ \lambda_2 = \dfrac{1}{\lambda_1} $.

In [1]:
# Import numpy
import numpy as np

# Import function which re-orients the vector field
from ipynb.fs.defs.orient_vectorfield import _orient_vectorfield

In [2]:
def _scaling_vectorfield_incompressible(X, Y, x, x_prime, vector_field, Interp_eig):
    '''
    Scaling of vectorfield turns tensorlines singularities into fixed points for incompressible vector fields.
    
    Parameters:
        X:               array (Ny, Nx), X-meshgrid
        Y:               array (Ny, Nx), Y-meshgrid 
        x:               array (2, Npoints), position (#Npoints = Number of initial conditions)
        x_prime:         array (2, Npoints), eigenvector at 'x'
        vector_field:    array (Ny, Nx, 2), eigenvector field over domain domain.
        interp_eig:      Interpolant-object for eigenvalue field
    
    Returns:
        rescaled_vector: array (2, Npoints), rescaled version of eigenvector. 
                         If the point is outside of the defined domain, then 'None' is returned
    '''
    
    vx, vy = _orient_vectorfield(X, Y, x, vector_field)
        
    if vx is not None:
        
        # compute lambda_2
        lambda_max = Interp_eig(x[1], x[0])[0][0]
        
        # if lambda_2 == 0 --> stop integration. 
        # This happens in regions close to the boundary, where the incompressibility condition is not satisfied anymore.
        if lambda_max == 0:
            return None
        
        # assuming incompressibility 
        lambda_min = 1/lambda_max
        
        # transform singularities to fixed points
        alpha = ((lambda_max-lambda_min)/(lambda_max+lambda_min))**2
        
        # rescalin
        scaling = np.sign(vx*x_prime[0]+vy*x_prime[1])*alpha
            
        rescaled_vector = scaling*np.array([vx, vy]) # array
    
        return rescaled_vector # array
        
    else:
        return None

In [ ]:
def _scaling_vectorfield_compressible(X, Y, x, x_prime, vector_field, Interp_eig, Interp_eig_opposite):
    '''
    Scaling of vectorfield turns tensorlines singularities into fixed points for compressible vector fields.
    
    Parameters:
        X:                   array (Ny, Nx), X-meshgrid
        Y:                   array (Ny, Nx), Y-meshgrid 
        x:                   array (2, Npoints), position (#Npoints = Number of initial conditions)
        x_prime:             array (2, Npoints), eigenvector at 'x'
        vector_field:        array (Ny, Nx, 2), eigenvector field over domain domain.
        interp_eig:          Interpolant-object for eigenvalue field
        interp_eig_opposite: Interpolant-object for opposite eigenvalue field
    
    Returns:
        rescaled_vector: array (2, Npoints), rescaled version of eigenvector. 
                         If the point is outside of the defined domain, then 'None' is returned
    '''
    
    vx, vy = _orient_vectorfield(X, Y, x, vector_field)
    
    if vx is not None:
        
        # compute lambda max
        lambda_max = Interp_eig(x[1], x[0])[0][0]
        
        # compute lambda min
        lambda_min = Interp_eig_opposite(x[1], x[0])[0][0]
        
        # Stop integration if there is no longer attraction/repulsion
        if (lambda_max-np.sign(lambda_max)<= 0):
            return None
        
        # transform singularities to fixed points
        alpha = (abs(lambda_max-lambda_min)/(lambda_max+lambda_min))**2
        
        # rescaling
        scaling = np.sign(vx*x_prime[0]+vy*x_prime[1])*alpha
            
        rescaled_vector = scaling*np.array([vx, vy]) # array
    
        return rescaled_vector # array
        
    else:
        return None

# References

[1] Tchon, K. F., Dompierre, J., Vallet, M. G., Guibault, F., & Camarero, R. (2006). Two-dimensional metric tensor visualization using pseudo-meshes. Engineering with Computers, 22(2), 121-131.

[2] Farazmand, M., & Haller, G. (2012). Computing Lagrangian coherent structures from their variational theory. Chaos: An Interdisciplinary Journal of Nonlinear Science, 22(1), 013128.